<a href="https://colab.research.google.com/github/Jokeren/triton-samples/blob/main/Triton_Tools_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

### Install PyTorch


In [1]:
!pip uninstall -y torch torchvision torchaudio
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121

Found existing installation: torch 2.4.0+cu121
Uninstalling torch-2.4.0+cu121:
  Successfully uninstalled torch-2.4.0+cu121
Found existing installation: torchvision 0.19.0+cu121
Uninstalling torchvision-0.19.0+cu121:
  Successfully uninstalled torchvision-0.19.0+cu121
Found existing installation: torchaudio 2.4.0+cu121
Uninstalling torchaudio-2.4.0+cu121:
  Successfully uninstalled torchaudio-2.4.0+cu121
Looking in indexes: https://download.pytorch.org/whl/nightly/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 784.2/784.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Alternative: Install Triton from source

In [ ]:
!git clone https://github.com/triton-lang/triton.git
%cd triton/python/

!pip install ninja cmake wheel pybind11; # build-time dependencies
!pip install -e .

Cloning into 'triton'...
remote: Enumerating objects: 140194, done.
remote: Counting objects: 100% (1707/1707), done.
remote: Compressing objects: 100% (874/874), done.
^C
[Errno 2] No such file or directory: 'triton/python/'
/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 19.6 MB/s eta 0:00:00
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


### Import packages

In [ ]:
import torch
import triton
import triton.language as tl

# Examples

### Example 1: Multi-dimensional array add

In [ ]:
@triton.jit
def add_kernel(x_ptr, y_ptr, z_ptr, dim0, dim1, BLOCK_DIM0: tl.constexpr, BLOCK_DIM1: tl.constexpr):
    pid_x = tl.program_id(axis=0)
    pid_y = tl.program_id(axis=1)
    block_start = pid_x * BLOCK_DIM0 * dim1 + pid_y * BLOCK_DIM1
    offsets = block_start + tl.arange(0, BLOCK_DIM0)[:,None] * dim1 + tl.arange(0, BLOCK_DIM1)[None, :]
    masks = (tl.arange(0, BLOCK_DIM0)[:,None] < dim0) & (tl.arange(0, BLOCK_DIM1)[None, :] < dim1)
    x = tl.load(x_ptr + offsets, mask=masks)
    y = tl.load(y_ptr + offsets, mask=masks)
    output = x + y
    tl.store(z_ptr + offsets, output, mask=masks)

# Interpreter

### tl.device_print

# Proton

## Example 1:

## Example 2:

## Example 3:

## Example 4:

## Example 5:

## Example 6: Instruction Sampling

## Example 7: GPT-Fast